<a href="https://colab.research.google.com/github/vipulgote1999/Assignment_GIBots_Pune/blob/main/Train_NN_GiBots_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger


In [ ]:
wandb.login()

wandb: Currently logged in as: vipulgote4 (existence). Use `wandb login --relogin` to force relogin


True

In [ ]:
import pandas as pd
import numpy as np
import os, time,json,sys,random,warnings
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler
import keras

In [ ]:
warnings.simplefilter("ignore")
random.seed(42)
np.random.seed(42)
pd.set_option('display.max_columns', None)

In [ ]:
!cp "/content/drive/MyDrive/Assignment data/GI Bots/DATA  Scientist Assignment.zip" /content/

In [ ]:
!unzip "/content/DATA  Scientist Assignment.zip"

Archive:  /content/DATA  Scientist Assignment.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train =pd.read_csv("/content/train.csv")
train_labels = pd.read_csv("/content/trainLabels.csv")
train_labels.drop("id", axis=1,inplace=True)
test_data = pd.read_csv("/content/test.csv")
test_data.columns = train.columns

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  #'mean', 'median', or 'most_frequent'
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# bool_transformer = Pipeline(steps=[
#     ('imputer',KNNImputer())
# ])

# here we are preprocessing  numerical and categorical features
preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, make_column_selector(dtype_include=np.number)),
        ('cat', categorical_transformer, make_column_selector(dtype_include=object))
    ], remainder='passthrough',n_jobs=-1)

X = train
y=train_labels[:9999]

In [ ]:
# X_train = pd.DataFrame(preprocessor.fit_transform(train).toarray())
# # y_train = preprocessor.transform(y_train)

# X_train.to_csv("Processed_train.csv")

# Train a model


In [ ]:
!pip install -qq tensorflow_addons numba

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow_addons.metrics import F1Score
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# preprocessor.fit(X_train)
X_train = preprocessor.fit_transform(X_train)
# y_train = preprocessor.fit_transform(y_train)

In [ ]:
X_train.shape, y_train.shape

((7999, 29067), (7999, 33))

In [ ]:
X_train

<7999x29067 sparse matrix of type '<class 'numpy.float64'>'
	with 1167854 stored elements in Compressed Sparse Row format>

In [ ]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.exp(-0.1)

In [ ]:
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

model = Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],),sparse=True))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
model.add(Dense(33, activation='sigmoid'))

adamw_opt = keras.optimizers.AdamW(learning_rate=0.01)
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy",# optimizer=adamw_opt,
              optimizer=opt,
              metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(patience=10,restore_best_weights=True)
lr_callback = keras.callbacks.LearningRateScheduler(scheduler)

ReduceLROnPlatea_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                              patience=3, min_lr=0.0001)

run = wandb.init(
    project = "GiBots-finetune"
)
# model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
model.fit(X_train.todense(), y_train.values, epochs=100, batch_size=64,validation_split=0.2, callbacks=[early_stopping,ReduceLROnPlatea_lr,WandbMetricsLogger(log_freq=10,)])

run.finish()

Epoch 1/100
100/100 [==============================] - 9s 20ms/step - loss: 0.0794 - accuracy: 0.6288 - val_loss: 0.0475 - val_accuracy: 0.7144 - lr: 0.0100
Epoch 2/100
100/100 [==============================] - 1s 12ms/step - loss: 0.0368 - accuracy: 0.7811 - val_loss: 0.0421 - val_accuracy: 0.7700 - lr: 0.0100
Epoch 3/100
100/100 [==============================] - 1s 13ms/step - loss: 0.0222 - accuracy: 0.8645 - val_loss: 0.0380 - val_accuracy: 0.7837 - lr: 0.0100
Epoch 4/100
100/100 [==============================] - 2s 17ms/step - loss: 0.0161 - accuracy: 0.8972 - val_loss: 0.0410 - val_accuracy: 0.7713 - lr: 0.0100
Epoch 5/100
100/100 [==============================] - 2s 16ms/step - loss: 0.0128 - accuracy: 0.9115 - val_loss: 0.0404 - val_accuracy: 0.7694 - lr: 0.0100
Epoch 6/100
100/100 [==============================] - 2s 17ms/step - loss: 0.0106 - accuracy: 0.9176 - val_loss: 0.0387 - val_accuracy: 0.7819 - lr: 0.0100
Epoch 7/100
100/100 [==============================] - 1s 

batch/accuracy,▁▃▃▆▆▇▇▇▇▇██▇█▇█████████████████████████
batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,██████████████▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▄▆▇▇▇███████████
epoch/epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
epoch/learning_rate,█████▂▂▂▂▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/lr,██████▂▂▂▂▁▁▁▁▁▁▁
epoch/val_accuracy,▁▅▆▅▅▆▇█▇████████
epoch/val_loss,█▅▂▄▄▃▁▁▂▂▂▂▂▂▂▂▃


## To View Training Dashboard click on below link:
https://wandb.ai/existence/GiBots-finetune

In [ ]:
test_data

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,3,0.895500,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,1,0.945032,0.471747,NO,NO,NO,NO,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,0.832679,0.049834,0.945938,0.317427,0.480308,NO,NO,YES,NO,NO,4,0.804893,3,7,0.966777,0.0,0.84,4672,3311,NO,NO,NO,3,0.838115,0.471318,6Wv/YGbS0KDFv2UrATvlJcVtCjJOgnVQzWuuF5Ltv2k=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,dBSc/QZM58O6miC4ULLhY0C4S6WIZLwy2oERlRo7Iaw=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,0,0.725763,17,7,0.876992,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
1,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,YES,YES,YES,4,0.536996,2,10,0.979821,0.0,1.00,1263,892,NO,NO,NO,8,0.557175,0.693587,NO,NO,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.00,1263,892,NO,NO,NO,7,0.526906,0.693587,0+HOWI7C643OXIZ7LdoeNFzA7A0MkSH//diziszbII8=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.00,1263,892,NO,NO,NO,7,0.526906,0.693587,eI6XNOFa+S9J5wTICUEas3M6YE4TJ6rSTCo2NvqM16s=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.00,1263,892,NO,NO,NO,7,0.526906,0.693587,1.415919,0.000000,1.000000,0.000000,0.703088,NO,NO,YES,YES,YES,0,0.466368,9,10,0.919283,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
2,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0,0.000000,0,0,0.000000,0.0,0.00,0,0,NaN,NaN,NaN,0,0.000000,0.000000,NO,NO,NO,NO,+X/rq+4qeKy2UVQN5xLQHSbri0Vdd/mYdG7WH/Kn+Pg=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,0.646703,0.059891,0.921960,0.531760,0.386558,NO,NO,NO,NO,NO,2,0.771325,6,9,0.939504,0.0,0.88,4672,3306,NO,NO,NO,3,0.831821,0.377568,nxOTdokWdl63qtAt4sgkSvyREdGu+ht1Iw4pcCaMi64=,YES,NO,QuSoZ63+jzXv3Ux1Y2LB+aSj7QCdPbrup3JSBj6Dmx8=,+yhSY//Hpg7u0bSA7NYmcmRFgv3bF4Tw3BMHrBqaTtA=,0.646703,0.059891,0.921960,0.531760,0.427226,YES,NO,YES,NO,YES,3,0.833031,1,7,0.992136,0.0,0.93,4672,3306,YES,NO,YES,1,0.840895,0.419949,B+EJpnEbkYtLnwDQYN1dP1rcfnoCnxAjKLYwQZE07Ew=,NO,NO,JA7Y7rY1wqr46lhqu5scuWov+Xw4neBTlPOPObhG0V8=,PjdztVAv+7nhxs8/+uAT0IRL7OcEHhLwHZ0IDTVbqcs=,0.646703,0.059891,0.921960,0.531760,0.416096,NO,NO,NO,NO,NO,13,0.161525,25,3,0.749546,0.0,0.86,4672,3306,NO,NO,NO,1,0.411978,0.405394,0.646703,0.059891,0.921960,0.531760,0.413099,YES,NO,NO,NO,NO,2,0.823351,5,3,0.952813,0.0,0.870000,4672,3

In [ ]:
prep_test_data = preprocessor.transform(test_data)
prep_test_data

<1999x29067 sparse matrix of type '<class 'numpy.float64'>'
	with 286132 stored elements in Compressed Sparse Row format>

In [ ]:
prep_test_data.shape

(1999, 29067)

In [ ]:
preds = model.predict(prep_test_data)
prep_sub_ids = [str(int(num)) + '_y' + str(int(i)) for num in test_data.id for i in range(1, 34)]
flatten_pred_list = preds.flatten()

submission_df = pd.DataFrame([prep_sub_ids,flatten_pred_list]).T
submission_df.columns = ["id","pred"]
# submission_df.head()

submission_df.to_csv("NN_model_submission_file.csv",index=False)

63/63 [==============================] - 0s 7ms/step
